# �� Smart Cage - Dataset Collection

**Samsung Innovation Campus - Phase 3**

Notebook ini untuk mengumpulkan dataset dari ESP32 via MQTT

**Target:**
- Minimal 150 baris data
- 3 Label: Ideal, Panas, Dingin
- Format: CSV dengan kolom temperature, humidity, label

## 📦 Step 1: Install Dependencies

In [ ]:
!pip install paho-mqtt pandas

: 

## 📚 Step 2: Import Libraries

In [ ]:
import json
import pandas as pd
import paho.mqtt.client as mqtt
from datetime import datetime
import time

print("✅ Libraries imported!")

## ⚙️ Step 3: MQTT Configuration

In [ ]:
MQTT_BROKER = "broker.hivemq.com"
MQTT_PORT = 1883
TOPIC_DATA = "final-project/Mahasiswa-Berpola-Pikir/smartcage/data"

print(f"Broker: {MQTT_BROKER}")
print(f"Topic: {TOPIC_DATA}")

## 💾 Step 4: Initialize Data Storage

In [ ]:
data_collected = []
target_samples = 150

print(f"Target: {target_samples} samples")
print("Ready to collect data!")

## 🎯 Step 5: Auto-Labeling Function

In [ ]:
def auto_label(temperature):
    """Auto-label berdasarkan threshold suhu"""
    if temperature > 35:
        return "Panas"
    elif temperature < 28:
        return "Dingin"
    else:
        return "Ideal"

# Test
print(f"Test 40°C: {auto_label(40)}")
print(f"Test 30°C: {auto_label(30)}")
print(f"Test 25°C: {auto_label(25)}")

## 📡 Step 6: MQTT Callbacks

In [ ]:
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("✅ Connected to MQTT broker")
        client.subscribe(TOPIC_DATA)
        print(f"📍 Subscribed to {TOPIC_DATA}")
    else:
        print(f"❌ Connection failed: {rc}")

def on_message(client, userdata, msg):
    global data_collected
    try:
        payload = msg.payload.decode("utf-8")
        data = json.loads(payload)
        
        temp = float(data.get("temp", 0))
        humidity = float(data.get("humidity", 0))
        
        # Auto-label
        label = auto_label(temp)
        
        # Tambahkan ke dataset
        entry = {
            "temperature": temp,
            "humidity": humidity,
            "label": label
        }
        data_collected.append(entry)
        
        # Progress
        count = len(data_collected)
        print(f"[{count}/{target_samples}] T={temp:.1f}°C | H={humidity:.1f}% | {label}")
        
        # Auto-stop saat cukup
        if count >= target_samples:
            print(f"\n✅ Target {target_samples} samples tercapai!")
            client.disconnect()
    except Exception as e:
        print(f"❌ Error: {e}")

print("✅ Callbacks defined!")

## �� Step 7: Start Collection

In [ ]:
client = mqtt.Client(client_id=f"DataCollector_{int(time.time())}")
client.on_connect = on_connect
client.on_message = on_message

print(f"🔌 Connecting to {MQTT_BROKER}...")
client.connect(MQTT_BROKER, MQTT_PORT, 60)

print("🎧 Listening for data from ESP32...")
print("⏱️ Collection will auto-stop at 150 samples\n")

# Run loop until disconnect
client.loop_forever()

## 💾 Step 8: Save to CSV

In [ ]:
if len(data_collected) > 0:
    df = pd.DataFrame(data_collected)
    
    filename = f"smart_cage_dataset_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv"
    df.to_csv(filename, index=False)
    
    print("=" * 50)
    print("💾 DATASET SAVED!")
    print("=" * 50)
    print(f"📁 Filename: {filename}")
    print(f"📊 Total Rows: {len(df)}")
    print("\n📋 Label Distribution:")
    print(df["label"].value_counts())
    print("\n📈 Preview:")
    display(df.head(10))
    print("\n📊 Statistics:")
    display(df.describe())
else:
    print("⚠️ No data collected!")